# Exercise 1: PID
In this exercise, you'll practice tuning PID constants for a straightforward system. Skip the next few blocks, down to the "Beginning of Exercise" heading (but remember to run each block in sequence, by hitting the play button).

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from matplotlib import animation, rc

In [0]:
def dynamics_simple(x, u):
  return -x + u

def dynamics_modified(x, u):
  return -x + 0.8 * u

In [0]:
state = 1.0

def simulate(x0, dynamics, control, ts, goal=None, animate=False):
  x = x0
  xs = []
  us = []
  for i, t in enumerate(ts[:-1]):
    u = control(x)
    xs = xs + [x]
    us = us + [u]

    dt = ts[i+1] - t
    x = x + dt * dynamics(x, u)
  
  # Get x[t]
  xs = xs + [x]

  nx = len(xs)
  xs = np.array(xs).reshape(nx)
  us = np.array(us).reshape(nx - 1)

  fig = plt.figure()
  xlim = (np.min(ts), np.max(ts))
  if goal is not None:
    ylim = (min(np.min(xs).item(), np.min(us).item(), np.min(goal)) - 0.1, max(np.max(xs).item(), np.max(us).item(), np.max(goal)) + 0.1)
  else:
    ylim = (min(np.min(xs).item(), np.min(us).item()) - 0.1, max(np.max(xs).item(), np.max(us).item()) + 0.1)
  ax = fig.add_subplot(111, autoscale_on=False, xlim=xlim, ylim=ylim)
  ax.grid()

  if goal is not None:
    ax.plot(ts, goal, '.', lw=1)

  ax.plot(ts, xs, '-', label='x')
  ax.plot(ts[:-1], us, '-', label='u')
  ax.legend()
  plt.show()

## Beginning of Exercise

### 1. Open-Loop
First, get a feel for the system by running it in open-loop mode. Modify the code below to return a constant value. Try to match the goal velocity.

_Hint: initialize the notebook by clicking the cell below and pressing "Runtime > Run Before" in the menu._

_Hint: Run the simulation by pressing the play button_

In [0]:
 Lgoal = 2

def control_open_loop(x):
  return 0

simulate(0.0, dynamics_simple, control_open_loop, np.linspace(0, 4, 100), [goal] * 100)

Now, try your same controller on a modified dynamics (in this case, I modelled in a 20% efficiency loss - not unreasonable for a mechanical system).

_(You don't need to change anything, just run the next cell)_

In [0]:
simulate(0.0, dynamics_modified, control_open_loop, np.linspace(0, 4, 100), [goal] * 100)

### 2. Adding Feedback
Next, add bang-bang feedback to the controller. When `x < goal`, apply full power of `u = 5`. Otherwise, let `u = 0`.

In [0]:
goal = 2

def control_bang_bang(x):
  return ...

simulate(0.0, dynamics_simple, control_bang_bang, np.linspace(0, 4, 100), [goal] * 100)

### 3. Proportional Response
Write a proportional controller. As a reminder, you want to calculate the error and then set `u = k_p*e`.

When you're done, run your controller on the modified dynamics (the second box) and see how it reacts.

In [0]:
goal = 2

def control_proportional(x):
  # TODO: Find a useful value of k_p, calculate e, and calculate u from k_p and e
  k_p = 0
  e = 0
  return 0

simulate(0.0, dynamics_simple, control_proportional, np.linspace(0, 4, 100), [goal] * 100)

In [0]:
simulate(0.0, dynamics_modified, control_open_loop, np.linspace(0, 4, 100), [goal] * 100)

### 4. PID(F) Controller
Finish up by writing a full PID controller (plus feedforwards/open-loop, if you have time). See what happens if you change the constants.

_Hint: start with your previous P controller. Then, calculate the derivative (by finite differences, `(e - e_last) / dt`) and integral (by summing `e*dt`). The `dt` for this simulation is 0.01._

In [0]:
goal = 2

integral = 0
last_error = 0

def control_pid(x):
  # TODO: Write a PID controller
  k_p = 0
  k_i = 0
  k_d = 0

  return 0

simulate(0.0, dynamics_simple, control_pid, np.linspace(0, 4, 100), [goal] * 100)

Run your PID controller on the modified dynamics. What happens?

In [0]:
simulate(0.0, dynamics_modified, control_pid, np.linspace(0, 4, 100), [goal] * 100)

# Exercise 2: Modeling a motor
The dynamics function I wrote up earlier was totally fudged. Made up. Garbage. Let's try writing up a real system.

### Modeling a motor
Let's try modeling a brushed DC motor. Take a look at the equations from the slides if you need a refresher.

In [0]:
Kt = 1.0
Kv = 1.0
gear_ratio = 2.0
moment_inertia = 0.08
motor_resistance = 0.02

def dynamics(velocity, voltage):
  # Calculate angular acceleration of the output shaft given voltage and angular velocity
  pass

### 2. State-space form
Next, let's model the dynamics just using matrices. The structure has been filled in, you just need to put in the constants.

In [0]:
A = np.array([0.0])
B = np.array([0.0])

def state_space_dynamics(x, u):
  return A @ x + B @ u

### 3. Adding in position
Finally, write a model with two state variables: `x = [position; velocity]`.

In [0]:
A = np.array([[0.0, 0.0],
              [0.0, 0.0]])
B = np.array([[0.0],
              [0.0]])

def state_space_dynamics(x, u):
  return A @ x + B @ u

## 4. Visualizing state-space plots
We can use a _phase diagram_ to visualize a system, in addition to simply plotting the variables regularly.

Run the box below to simulate the system with a step control input for a short period of time.
(There are no changes you need to make)

In [0]:
def simulate_ss(x0, ts, controller, goal=None):
  x = x0

  x0s = []
  x1s = []
  r0s = []
  r1s = []
  us = []

  for t in ts:
    u = controller(x, t)
    x0s.append(x[0])
    x1s.append(x[1])
    if goal is not None:
      p, v = goal(t)
      r0s.append(p)
      r1s.append(v)
    us.append(u[0])
    x += 0.01 * state_space_dynamics(x, u)

  fig, axs = plt.subplots(1, 2, figsize=(12, 6))

  axs[0].plot(ts, x0s)
  axs[0].plot(ts, x1s)
  axs[0].plot(ts, us)
  if goal:
    axs[0].plot(ts, r0s)
    axs[0].plot(ts, r1s)

  axs[0].set_xlabel('time')
  axs[0].legend(['pos', 'vel', 'input'] + (['goal pos', 'goal vel'] if goal else []))

  min_x0, max_x0 = np.min(x0s), np.max(x0s)
  min_x1, max_x1 = np.min(x1s), np.max(x1s)
  min_u, max_u = np.min(us), np.max(us)
  X0, X1 = np.meshgrid(np.linspace(min_x0 - 1, max_x0 + 1, 10), np.linspace(min_x1 - 1, max_x1 + 1, 10))
  dX = np.zeros((2, X0.shape[0], X0.shape[1]))
  for i in range(X0.shape[0]):
    for j in range(X0.shape[1]):
      X = np.array([X0[i, j], X1[i, j]])
      dX[:, i, j] = state_space_dynamics(X, controller(X, 1000))

  axs[1].quiver(X0, X1, dX[0], dX[1])
  axs[1].plot(x0s, x1s)
  axs[1].set_aspect('equal', 'box')
  axs[1].set_xlabel('position')
  axs[1].set_ylabel('velocity')

  plt.show()
simulate_ss(np.array([0, 0.]), np.linspace(0, 5, 500), lambda x, t: np.array([3.0 if t < 1.5 else 0.0]))

# Exercise 3: Pole Placement
Let's try to move the motor to a specific _position_ instead of a velocity. We'll use pole placement to design our controller.

Because this is a 2-dimensional system, there are going to be exactly two poles. Remember that they can be either purely real or complex, and that solutions (the actual result of the simulation) will look like $ae^{\lambda_1t}+be^{\lambda_2t}$. What values of $\lambda$ will tend to drive the system towards zero?

Try a few different values for poles! Also, try making the poles include complex numbers.

In [0]:
from scipy.signal import place_poles

# Use place_poles(A, B, poles) to design a controller. This will create
# a system dx/dt = (A-BK)x
fsfb = ... # Place poles
K = fsfb.gain_matrix

simulate_ss(np.array([5., 0]), np.linspace(0, 5, 500), lambda x, t: -(K @ x))

### Note on complex numbers
If you haven't taken a linear algebra class (or if you have), you may be confused. How would a purely real matrix have imaginary eigenvalues? It turns out that this happens when the matrix _rotates_ the plane instead of just squishing and stretching it. Think about how this corresponds to what you saw above.

You might have also noticed that the code above requires complex numbers in _pairs_ - if you want a pole at $a+bi$, you also need to have one at $a-bi$. The exact explanation for this takes a little explaining, but you can think of it (sort of) like this: when you multiply all of the eigenvalues together, you need to get a real number (this is the _determinant_ of the matrix). The numbers that act like this are the complex conjugates.

## What if we want to get somewhere other than zero?
How could we design a controller that gets to a nonzero position? (I'll give you a hint: error).

In [0]:
goal = np.array([3.0, 0])
def controller(x, t):
  return -K @ x

simulate_ss(np.array([5., 0]), np.linspace(0, 5, 500), controller)

# Exercise 4: The Real World
## Control Limits
What happens when we add in control limits? Give it a try...

Can you find a set of poles that makes the system stable, even with control limits? Try to think about why it isn't stable in the first place, even though the poles should be negative.

In [0]:
# This is an unstable system...let's stabilize it.
A = np.array([[0, 1],
              [-1, 1.5]])

# These poles are both negative, that should do it - right?
K = place_poles(A, B, [-2, -3]).gain_matrix

def controller(x, t):
  # Clip to +-5
  return np.clip(-K @ x, -5, 5)

simulate_ss(np.array([2., 3]), np.linspace(0, 8, 800), controller)

## Motion Profiling

We want to move more smoothly from the start to the goal. The start of a simple trapezoid profile has been written - finish it up!

There are lots of ways to implement a trapezoidal profile, but the simplest is to calculate the start time of each of the three segments. Then, you can just interpolate when you're calculating the actual profile.

In [0]:
class TrapezoidProfile:
  def __init__(self, a_max, v_max, goal):
    """
    Assumptions:
     - a_max and v_max are positive
     - goal is positive
     - you start at zero velocity and zero position
     - you will have enough time to ramp all the way up and all the way down
    """
    self.a_max = a_max
    self.v_max = v_max
    self.goal = goal
    # TODO: Calculate the time to accelerate to maximum velocity, and the distance
    # you travel during this period
    self.ramp_time = ...
    self.ramp_distance = ...

    # TODO: Calculate the time to ramp down to zero.
    self.ramp_down_time = ...
    self.ramp_down_distance = ...

    # Calculate the distance left
    self.constant_vel_distance = goal - self.ramp_distance - self.ramp_down_distance
    self.constant_vel_time = self.constant_vel_distance / v_max

  def calculate(self, t):
    position = 0
    velocity = 0
    if t < self.ramp_time:
      # Acceleration

      # The time into this segment (since the beginning of the constant phase)
      t0 = t

      # TODO: Fill these out
      position = ...
      velocity = ...
    elif t < self.ramp_time + self.constant_vel_time:
      # Constant speed

      # The time into this segment (since the beginning of the constant phase)
      t1 = t - self.ramp_time

      # Position at the beginning of this segment
      d1 = self.ramp_distance

      # TODO: Fill these out
      position = d1 + ...
      velocity = ...
    elif t < self.ramp_time + self.constant_vel_time + self.ramp_down_time:
      # Deceleration
      
      # The time into this segment (since the beginning of the deceleration phase)
      t2 = t - self.ramp_time - self.constant_vel_time

      # Position at the beginning of this segment
      d2 = self.ramp_distance + self.constant_vel_distance
      
      # TODO: Fill these out
      position = d2 + ...
      velocity = ...
    else:
      # We're done
      position = self.goal
      velocity = 0
    return position, velocity

profile = TrapezoidProfile(1.0, 2.0, 8.0)
ts = np.linspace(0, 8, 100)

xs = []
vs = []
for t in ts:
  p, v = profile.calculate(t)
  xs.append(p)
  vs.append(v)

plt.plot(ts, xs)
plt.plot(ts, vs)
plt.legend(['x', 'v'])
plt.show()

## Motion Profiled Control
Finally, use the profile as a setpoint for your controller. What happens?

In [0]:
# This is an unstable system...let's stabilize it.
A = np.array([[0, 1],
              [0, -0.5]])

# These poles are both negative, that should do it - right?
K = place_poles(A, B, [-2, -3]).gain_matrix

profile = TrapezoidProfile(3.0, 2.0, 5.0)

def controller(x, t):
  p, v = profile.calculate(t)
  goal = np.array([p, v])
  # Clip to +-5
  return np.clip(K @ (goal - x), -5, 5)

simulate_ss(np.array([0., 0]), np.linspace(0, 5, 500), controller, lambda t: profile.calculate(t))